# Report Basics of Mobile Robotics - 2024

## Table of Contents
0. [Introduction](#Introduction)
1. [Setup](#Partie-1-:-Setup)
    - 1.1 : Physical setup
    - 1.2 : Code setup
2. [Initial tuning](#Partie-2-:-Initial-tuning)
    - 2.1 : Wheel speed (differential)
3. [Vision](#Partie-2-:-Vision)
    - 3.1 : Dynamic lighting adaptation
    - 3.2 : Aruko markers
        - 3.2.1 : Thymio and goal
    - 3.2 : Map resizing
4. [Kalmann](#Partie-3-:-Kalmann)
    - 4.1 : Theory
    - 4.2 : Implementation
5. [Global path planning](#Partie-4-:-Global-path-planning)
    - 5.1 : Dijkstra
6. [Local navigation](#Partie-5-:-Local-navigation)
    - 6.1 : Local obstacle detection
    - 6.2 : Potential fields
7. [Motion control](#Partie-5-:-Motion-control)
    - 7.1 : Differential drive
    - 7.2 : P-conntroler
8. [Demonstrations](#Partie-5-:-emonstrations)
9. [Conclusion](#IConclusion)
---



## Introduction
Authors : Tifaine Mezencev, Julien, Zhuoran, Christy

---



## 1 : Setup

### 1.1 : Physical setup

```Explication des données utilisées.```

A small but determined vessel navigating uncharted waters, its sights set on a glittering treasure at the far edge of the map. Along the way, it must skillfully avoid perilous islands and evade the watchful patrols of navy ships.

In this project, we implement global and local navigation for the Thymio robot. The goal is to enable the robot to plot an optimal course toward its destination while dynamically avoiding both static and unexpected obstacles. The key features of our implementation include:

- Accurate map creation and feature localization using ArUco markers
- Global pathfinding using Dijkstra's algorithm
- Seamless connection to the Thymio robot for real-time path execution
- Path-following with dynamic adjustment for precision navigation
- Emergency handling, including "Thymio kidnapping" detection and recovery
- Local obstacle avoidance with a potential field method
- Robust navigation supported by a Kalman filter in the absence of camera input

With this project, we aim to demonstrate how a small robot can overcome big challenges, transforming obstacles into opportunities and bringing our vision of autonomous navigation to life!





### 1.2 : Code setup

## 2 : Initial tuning

### 2.1 Wheel speed (differential)

## 3: Vision

The Vision.py module has two tasks : run the camera and analyse the frames captured by the camera. This is why the Vision class is actually more of a wrapper around the functionnalities given by the two other classes defined in the vision file, appropriately named Analysis and CameraFeed. 


Let's dive into Analysis.
 
Given an image, the class has three tasks : pinpoint the thymio, the goal, and the 2D (black) obstacle in a consistent reference frame.  Of course doing this is easier said than done. A getter function of Vision will then be used to access these values and pass them on to the other modules.
The first step is defining what is in the map and what is outside the map.


### 3.1 Map

We first wanted to use color to differenciate the components of our problem and we painted our map in blue and obstacles in black. Specifically, we tried using a blue color mask to make out the map and pinpoint its four corners. This was relavitely inconsistent however, and we decided to use ArUco markers to make our life easier.

In [1]:
def detect_obstacle_corners(self, img):
    """
    Detects obstacle edges in the image, approximates them as polygons, 
    and keeps only those with an average color that resembles black (obstacles).
    Returns three outputs: the polygons, the binary obstacle mask, and the image with visualized polygons.
    """
    # 1. Convert the image to LAB color space for color analysis
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # 2. Calculate the histogram of the L component
    hist = cv2.calcHist([lab_img[:, :, 0]], [0], None, [180], [0, 180])  # 180 bins for L in the range [0, 180]
    signal = np.gradient(medfilt(hist.flatten(), 15))  # Median filtering to smooth the signal

    grad_threshold = -10
    highest = 125

    # 3. Search for the threshold in the signal
    crossing_index = -1
    for i in range(highest - 1, 1, -1):  # Start just below 'highest' and move backwards
        if signal[i] < grad_threshold:
            crossing_index = i
            break

    # # Find the threshold
    # # if crossing_index != -1:
    # #     print(f"The highest index where the signal crosses {grad_threshold} is {crossing_index}.")
    # # else:
    # #     print(f"No crossing found before index {highest}.")

    treshold = crossing_index

    # 4. Create a binary mask based on the threshold
    mask = lab_img[:, :, 0] < treshold
    obstacles = np.uint8(mask * 255)

    # 5. Process connected components
    COUNT = 3000  # Minimum size threshold for components
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(obstacles, connectivity=8)
    filtered_mask = np.zeros_like(obstacles)

    for i in range(1, num_labels):  # Ignore background (label 0)
        if stats[i, cv2.CC_STAT_AREA] >= COUNT:
            filtered_mask[labels == i] = 255

    # 6. Morphological operations to smooth the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))  # Structuring element for dilation
    filtered_mask = cv2.dilate(filtered_mask, kernel)
    filtered_mask = cv2.erode(filtered_mask, kernel)

    # 7. Find contours of the obstacles
    contours, _ = cv2.findContours(filtered_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 8. List to store the corners of the obstacles
    obstacle_corners = []

    # 9. Approximate the contours as polygons
    for contour in contours:
        epsilon = 0.02 * cv2.arcLength(contour, True)  # Approximation precision
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Add the corners of the polygons
        obstacle_corners.append(approx.reshape(-1, 2).tolist())

    # 10. Visualize the polygons with colored edges and circles at corners
    img_with_polygons = img.copy()

    # Assign a unique color for each polygon
    for i, polygon in enumerate(obstacle_corners):
        # Random color for each polygon (BGR format)
        color = np.random.randint(0, 256, 3).tolist()

        # Draw the corners as red circles
        for (x, y) in polygon:
            cv2.circle(img_with_polygons, (x, y), 5, (0, 0, 255), -1)  # Red circles at corners

        # Draw the polygon edges
        polygon_array = np.array(polygon, dtype=np.int32)
        cv2.drawContours(img_with_polygons, [polygon_array], -1, color, 2)  # Polygon edges with random color

    # 11. Return the results: obstacle corners, binary mask, and image with visualized polygons
    return obstacle_corners, filtered_mask, img_with_polygons

### 3.1 : Dynamic lighting adaptation

This function processes an image to detect obstacles, identify their edges, and approximate their shapes as polygons. It returns the polygons, a binary mask of the detected obstacles, and an image visualizing the polygons

What the function does :
- **Convert the image to LAB color space:** It uses the L (lightness) component to identify dark regions (potential obstacles).
- **Threshold the lightness component:** It creates a binary mask to isolate dark regions.
- **Filter small components:** Only retains regions that meet a size threshold.
- **Smooth the mask:** Applies morphological operations to clean up the mask.
- **Find contours and approximate them as polygons:** These represent the shapes of obstacles.
- **Visualize the polygons:** Draws the approximated polygons and their corners on the original image.

In [3]:
def detect_obstacle_corners(self, img):
    """
    Detects obstacle edges in the image, approximates them as polygons, 
    and keeps only those with an average color that resembles black (obstacles).
    Returns three outputs: the polygons, the binary obstacle mask, and the image with visualized polygons.
    """
    # 1. Convert the image to LAB color space for color analysis
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # 2. Calculate the histogram of the L component
    hist = cv2.calcHist([lab_img[:, :, 0]], [0], None, [180], [0, 180])  # 180 bins for L in the range [0, 180]
    signal = np.gradient(medfilt(hist.flatten(), 15))  # Median filtering to smooth the signal

    grad_threshold = -10
    highest = 125

    # 3. Search for the threshold in the signal
    crossing_index = -1
    for i in range(highest - 1, 1, -1):  # Start just below 'highest' and move backwards
        if signal[i] < grad_threshold:
            crossing_index = i
            break

    # # Find the threshold
    # # if crossing_index != -1:
    # #     print(f"The highest index where the signal crosses {grad_threshold} is {crossing_index}.")
    # # else:
    # #     print(f"No crossing found before index {highest}.")

    treshold = crossing_index

    # 4. Create a binary mask based on the threshold
    mask = lab_img[:, :, 0] < treshold
    obstacles = np.uint8(mask * 255)

    # 5. Process connected components
    COUNT = 3000  # Minimum size threshold for components
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(obstacles, connectivity=8)
    filtered_mask = np.zeros_like(obstacles)

    for i in range(1, num_labels):  # Ignore background (label 0)
        if stats[i, cv2.CC_STAT_AREA] >= COUNT:
            filtered_mask[labels == i] = 255

    # 6. Morphological operations to smooth the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))  # Structuring element for dilation
    filtered_mask = cv2.dilate(filtered_mask, kernel)
    filtered_mask = cv2.erode(filtered_mask, kernel)

    # 7. Find contours of the obstacles
    contours, _ = cv2.findContours(filtered_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 8. List to store the corners of the obstacles
    obstacle_corners = []

    # 9. Approximate the contours as polygons
    for contour in contours:
        epsilon = 0.02 * cv2.arcLength(contour, True)  # Approximation precision
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Add the corners of the polygons
        obstacle_corners.append(approx.reshape(-1, 2).tolist())

    # 10. Visualize the polygons with colored edges and circles at corners
    img_with_polygons = img.copy()

    # Assign a unique color for each polygon
    for i, polygon in enumerate(obstacle_corners):
        # Random color for each polygon (BGR format)
        color = np.random.randint(0, 256, 3).tolist()

        # Draw the corners as red circles
        for (x, y) in polygon:
            cv2.circle(img_with_polygons, (x, y), 5, (0, 0, 255), -1)  # Red circles at corners

        # Draw the polygon edges
        polygon_array = np.array(polygon, dtype=np.int32)
        cv2.drawContours(img_with_polygons, [polygon_array], -1, color, 2)  # Polygon edges with random color

    # 11. Return the results: obstacle corners, binary mask, and image with visualized polygons
    return obstacle_corners, filtered_mask, img_with_polygons

### 3.2 : ArUco markers
ArUco markers are binary patterns which are unique and have no symmetry. Due to this, they are often used in robotics and computer vision because seeing one markers makes it possible to completely determine the orientation of the camera relative to the marker (and its distance, if the size of the marker is known). They are part of a very convenient library which implements most of the code needed to detect the markers and the according translation and rotation transform matrices.

We have 6 markers : one per map corners, and two for the goal and thymio.

We wrote a function to find the center point and 2D orientation (defineed as angle between marker top edge and image X-axis). 
Yes, the aruco library already implements a rotation transform function, yes we forgot to read the docs before starting. But it's okay, it was a fun little geometry puzzle to solve.

### 3.2 : Map resizing
It is easier for us if the image is a perpendicular top-down view and everything that isn't part of the map is cropped out. To achieve this, we apply some visual transformations on the image. First we find the four corners of the map with our aruco markers We are lucky because opencv provides us with great tools to do this.

### 3.3 : Map resizing

### 3.7 Camera management 
We want the camera to always be running in the background and display the video feed continously (along with whichever debugging displays), independently of what the path planner, control or kalman modules are doing. 

This is why we use the ```threading``` library, which allows different processes to be run in parallel. The CameraFeed class inherits from the ```threading```.Thread class. 
Its core method is a while loop that captures a frame, finds its corners, resizes it, detects the Thyimio & goal and (if desired) draws and shows the debugging displays. 

At the start of the code, the begin() method of the Vision class initializes and runs a CameraFeed thread object, which will continue its work in the background for the rest of the program.

## 4 : Kalmann

### 4.1 : Theory

### 4.2 : Implementation

## 5 : Global path planning
 


### 5.1 : Dijkstra

## 6 : Local navigation

### 6.1 : Local obstacle detection

### 6.2 : Potential fields

## 7 : Motion control

### 7.1 : Differential drive

### 7.2 : P-conntroler

## 8 : Demonstrations

---
## 9 : Conclusion
---

